In [1]:
import re
import requests

In [9]:
def get_info_from_site(url):
    try:
        res = requests.get(url)
    except OSError:
        ...
    price = re.findall('<div class="price-new">(.*?)\\xa0₽<\/div>"', res.text)
    return price

In [10]:
item = 'https://www.perekrestok.ru/cat/366/p/smes-salatnaa-belaa-daca-tango-mix-65g-3451379'

get_info_from_site(item)

[]